In [6]:
%cd D:/uitds

D:\uitds


In [7]:
from PIL import Image

In [8]:
image_path = "data\\train-images\\00a001c55051fddfa8889e2372241f96917a04d76fe1be5e37fa3193ab6d4f61.jpg"
image = Image.open(image_path).convert('RGB')

In [9]:
import json
JSON_PATH = "data\\json/vimmsd-train-preprocess.json"

json_file = open(JSON_PATH, 'r')
json_data = json.load(json_file)

In [10]:
json_data["0"]

{'caption': 'Cô ấy trên mạng vs cô ấy ngoài đời  <icon> mặt cười </icon> ',
 'label': 'multi-sarcasm',
 'image': '8ae451edcd8ebf697f8763ece249115813149c55733bf8306b971210d48b15d3.jpg'}

In [11]:
# read txt file: data\train_val_group_split.txt
TXT_PATH = "data\\train_val_group_split.txt"

def read_train_test_file(name_file):
    # the first row is the number of train_idx and val_idx
    # the second row is the train_idx
    # the third row is the val_idx
    
    with open(name_file, 'r') as f:
        lines = f.readlines()
        num_train, num_val = int(lines[0].split()[0]), int(lines[0].split()[1])
        train_idx = list(map(int, lines[1].split()))
        val_idx = list(map(int, lines[2].split()))
    return num_train, num_val, train_idx, val_idx

num_train, num_val, train_idx, val_idx = read_train_test_file(TXT_PATH)

In [12]:
def process(label):
    return label

In [13]:
def prompt(caption):
    return f"<image>\nĐây là bức ảnh của 1 bài đăng trên các nền tảng xã hội, bài này có tiêu đề: {caption}.\n Hãy xác định bài đăng này có mang tính mỉa mai thuộc dạng nào trong 3 dạng sau: not-sarcasm, text-sarcasm, image-sarcasm. Nếu bài đăng có cả text-sarcasm và image-sarcasm thì trả về multi-sarcasm."

In [14]:
all_data = []
FOLDER_IMAGES = "data\\train-images"
IMAGE_PATH = "/kaggle/input/vintern-resize-images/resized_images/content/resized_images"
for id in train_idx:
    # image = Image.open(f"{FOLDER_IMAGES}\{json_data[str(id)]['image']}").convert('RGB')
    image_file = f"{IMAGE_PATH}/{id}.jpg"
    # width = image.size[0]
    # height = image.size[1]
    conversations = [
            {
                "from": "human",
                "value": prompt(json_data[str(id)]["caption"])
            },
            {
                "from": "gpt",
                "value": str(process(json_data[str(id)]["label"]))
            }
        ]
    data = {
        "id": id,
        "image": image_file,
        "conversations": conversations
    }
    all_data.append(data)

In [15]:
len(all_data)

9186

In [16]:
all_data

[{'id': 1,
  'image': '/kaggle/input/vintern-resize-images/resized_images/content/resized_images/1.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nĐây là bức ảnh của 1 bài đăng trên các nền tảng xã hội, bài này có tiêu đề: Người tâm linh giao tiếp với người thực tế  <icon> mặt cười </icon> .\n Hãy xác định bài đăng này có mang tính mỉa mai thuộc dạng nào trong 3 dạng sau: not-sarcasm, text-sarcasm, image-sarcasm. Nếu bài đăng có cả text-sarcasm và image-sarcasm thì trả về multi-sarcasm.'},
   {'from': 'gpt', 'value': 'not-sarcasm'}]},
 {'id': 2,
  'image': '/kaggle/input/vintern-resize-images/resized_images/content/resized_images/2.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nĐây là bức ảnh của 1 bài đăng trên các nền tảng xã hội, bài này có tiêu đề: Hình như Trăng hôm nay đẹp quá mọi người ạ!  <icon> Khuôn mặt cười toe toét với đôi mắt to </icon>   <icon> Khuôn mặt cười toe toét với đôi mắt to </icon> \n#canhco.\n Hãy xác định bài đăng này có m

In [17]:
### Write JSONL line
from tqdm import tqdm 
with open('data/jsonl/train_prompt_v6_kaggle.jsonl', 'w', encoding='utf-8') as f:
    for data in tqdm(all_data):
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

  0%|          | 0/9186 [00:00<?, ?it/s]

100%|██████████| 9186/9186 [00:00<00:00, 64736.24it/s]


In [18]:
import collections
collections.Counter([data["conversations"][1]["value"] for data in all_data])

Counter({'not-sarcasm': 5115,
         'multi-sarcasm': 3625,
         'image-sarcasm': 388,
         'text-sarcasm': 58})

In [19]:
# count the number of values in json_data without knowing before the value

from collections import Counter
Counter([json_data[str(i)]["label"] for i in train_idx])


Counter({'not-sarcasm': 5115,
         'multi-sarcasm': 3625,
         'image-sarcasm': 388,
         'text-sarcasm': 58})